In [11]:
import sqlite3
import pandas as pd

In [23]:
conn = sqlite3.connect('sqlite_db_pythonsqlite.db')
conn.row_factory = sqlite3.Row

In [24]:
c = conn.cursor()

In [68]:
# Q10: Produce a list of facilities with a total revenue less than 1000.
# The output of facility name and total revenue, sorted by revenue.
# Remember that there's a different cost for guests and members!

q10 = c.execute("""
        SELECT f.name AS facility, SUM(b.slots) AS slots, 
            SUM(CASE WHEN memid != 0 THEN slots * membercost
            ELSE slots * guestcost END) AS revenue
        FROM Bookings AS b
        LEFT JOIN Facilities AS f USING (facid)
        GROUP BY facility
        HAVING revenue < 1000
        ORDER BY revenue DESC
    """)
row = q10.fetchone()
df10 = pd.DataFrame(q10.fetchall())
df10.columns = row.keys()

df10

,facility,slots,revenue
0,Snooker Table,908,240
1,Table Tennis,830,180


In [94]:
# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

q11 = c.execute("""
        SELECT m1.surname, m1.firstname, (
            SELECT (m2.surname || ' ' || m2.firstname) 
            FROM Members AS m2 
            WHERE m1.recommendedby = m2.memid
            ) AS recommendedby
        FROM Members AS m1
        WHERE recommendedby IS NOT ''
        ORDER BY m1.surname, m1.firstname
    """)
row = q11.fetchone()
df11 = pd.DataFrame(q11.fetchall())
df11.columns = row.keys()

df11

,surname,firstname,recommendedby
0,Baker,Anne,Stibbons Ponder
1,Baker,Timothy,Farrell Jemima
2,Boothe,Tim,Rownam Tim
3,Butters,Gerald,Smith Darren
4,Coplin,Joan,Baker Timothy
5,Crumpet,Erica,Smith Tracy
6,Dare,Nancy,Joplette Janice
7,Genting,Matthew,Butters Gerald
8,Hunt,John,Purview Millicent
9,Jones,David,Joplette Janice


In [102]:
# Q12: Find the facilities with their usage by member, but not guests

q12 = c.execute("""
        SELECT f.name AS facility, 
            SUM(CASE WHEN memid != 0 THEN slots
            END) AS usage
        FROM Bookings AS b
        LEFT JOIN Facilities AS f USING (facid)
        GROUP BY facility
    """)
row = q12.fetchone()
df12 = pd.DataFrame(q12.fetchall())
df12.columns = row.keys()

df12

,facility,usage
0,Massage Room 1,884
1,Massage Room 2,54
2,Pool Table,856
3,Snooker Table,860
4,Squash Court,418
5,Table Tennis,794
6,Tennis Court 1,957
7,Tennis Court 2,882


In [109]:
# Q13: Find the facilities usage by month, but not guests

q13 = c.execute("""
        SELECT f.name AS facility, 
            strftime('%m', starttime) AS month,
            SUM(CASE WHEN memid != 0 THEN slots
            END) AS usage
        FROM Bookings AS b
        LEFT JOIN Facilities AS f USING (facid)
        GROUP BY facility, month
    """)
row = q13.fetchone()
df13 = pd.DataFrame(q13.fetchall())
df13.columns = row.keys()

df13

,facility,month,usage
0,Badminton Court,08,414
1,Badminton Court,09,507
2,Massage Room 1,07,166
3,Massage Room 1,08,316
4,Massage Room 1,09,402
5,Massage Room 2,07,8
6,Massage Room 2,08,18
7,Massage Room 2,09,28
8,Pool Table,07,110
9,Pool Table,08,303


In [110]:
conn.close()